In [1]:
import sys
sys.path.append("home/kelab/m6AAIpy2")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('home/kelab/iM6A/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
models

### Read data

In [2]:
import numpy as np
import pandas as pd

In [3]:
Fasta = pd.read_csv("mm10_Fasta_Non_Donor_Acceptor.csv")

In [4]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq
0,DUPD1,chr10,-,76797593,76818272,76797593,76818272,3,"76797593,76803554,76818072,","76797835,76803775,76818272,",...,242,5719,221,14497,ATGACATCTGGAGAAGTGAAGACAAGCCTCAAGAATGCCTACTCAT...,GGCGACGGCGCTGGACCGCTATAGGCTGCAGAAGGCGGGGTTCACG...,GTAAGAGACCCGCGgccccgccccgatcggccccgccccgccccgc...,GTAAGATCCTGGTTCACTGCGTCATGGGCCGCAGCCGGTCAGCCAC...,GTAAGAGACCCGCGGCCCCGCCCCGATCGGCCCCGCCCCGCCCCGC...,GGCGACGGCGCTGGACCGCTATAGGCTGCAGAAGGCGGGGTTCACG...
1,PSD2,chr5,+,139175405,139224051,139189025,139222059,15,"139175405,139188975,139192893,139193754,139197...","139175560,139189396,139193343,139193949,139197...",...,2196,2100,144,44206,ctcgctcagcctctccACATCGCGGCTCCGGCACCTGAAGGGAcgc...,GAGGAGCAACTGCGGTCTCATGAGAATAAGTTGAGGCAGCTGACTG...,GTGAGCCTTGGGGGACTGGATTGCAAAGCCCAGTGCCCTGCTCTTT...,AAAAGCCGTTATGAGACCTATATCCACCTCCTGGCTATGAAAATCA...,GTGAGCCTTGGGGGACTGGATTGCAAAGCCCAGTGCCCTGCTCTTT...,GAGGAGCAACTGCGGTCTCATGAGAATAAGTTGAGGCAGCTGACTG...
2,GNAQ,chr9,-,80331002,80646374,80336238,80646151,7,"80331002,80343429,80409378,80412435,80430531,8...","80336429,80343583,80409508,80412564,80430686,8...",...,5427,7000,154,302791,CGTCGGCCCAGACTATCCGCTCCCAccgcgcccccggcccacctgg...,AACCGAATGGAGGAAAGCAAGGCTCTCTTTAGAACAATTATCACAT...,GTAAGTCAGCACTCATACCACAGCTCAGTAGCTGTGAATAGGTTTT...,GACCCCAGAGAGATGCCCAGGCAGCCCGAGAATTCATTCTGAAGAT...,GTAAGTCAGCACTCATACCACAGCTCAGTAGCTGTGAATAGGTTTT...,AACCGAATGGAGGAAAGCAAGGCTCTCTTTAGAACAATTATCACAT...
3,NF1,chr17,+,29421944,29704695,29422327,29701173,57,"29421944,29483000,29486027,29490203,29496908,2...","29422387,29483144,29486111,29490394,29497015,2...",...,3665,13309,217,265560,AATCTCTAGCTCGCTCGCGCTCCCTCTCCCCGGGCCGTGGAAAGGA...,CAAACACAAATTCCAGACTATGCTGAGCTTATTGTTAAGTTTCTTG...,GTCAGTAAATGTGATCTTTATATGACTTTGAGCAACAATATAAGAC...,GAATCGACAAGGAGAACGTTGAACTCTCCCCTACCACTGGCCACTG...,GTCAGTAAATGTGATCTTTATATGACTTTGAGCAACAATATAAGAC...,CAAACACAAATTCCAGACTATGCTGAGCTTATTGTTAAGTTTCTTG...
4,YWHAZ,chr8,-,101930913,101965569,101932920,101961117,6,"101930913,101936182,101936362,101937143,101960...","101932980,101936278,101936526,101937267,101961...",...,2067,3202,96,29291,GCCCACTCCCACCGCCAGCTGGAACCCTGGGGACTACGACGTCCCT...,GCTTTTGATGAAGCCATTGCTGAACTTGATACATTAAGTGAAGAGT...,GTAAGTACATAACAGCATCATTAGTGATGAATGTGGTTGAAGAGGT...,TTGTGGACATCGGATACCCAAGGAGACGAAGCTGAAGCAGGAGAAG...,GTAAGTACATAACAGCATCATTAGTGATGAATGTGGTTGAAGAGGT...,GCTTTTGATGAAGCCATTGCTGAACTTGATACATTAAGTGAAGAGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,CILP,chr15,-,65488336,65503826,65489068,65502093,9,"65488336,65494209,65495699,65496605,65497624,6...","65491437,65494367,65495808,65496920,65497804,6...",...,3101,2772,158,9459,ACTGGACTGCTGGAGAGGCGAAGGCTTGGACCCCCAGCTGAGGAGT...,GTATCATAATGACACATTGCTGGATCCTTCCCTCTACAAGCATGAG...,GTAAGCCTGTCTGGGTCCCTGGGGCCCCTGGGTCTGCAACAGGGTT...,CATCTGATGAGACTCCTTGCAACCCAGTTCCTGAGAGCTATCTTAT...,GTAAGCCTGTCTGGGTCCCTGGGGCCCCTGGGTCTGCAACAGGGTT...,GTATCATAATGACACATTGCTGGATCCTTCCCTCTACAAGCATGAG...
3319,ZBTB14,chr18,-,5289017,5296039,5290856,5293245,4,"5289017,5293242,5293970,5295811,","5292203,5293326,5294000,5296039,",...,3186,1039,84,2713,GTGTTCACGGCCGGCTAGACATTCTGTGCTCGTGCAGGAGGAGGGC...,GTCTGGCGTGCTCCTGATCAGAGTTACTCTGATCTGAAGATTTTGG...,GTAACTATAACTTTGGATATTAAATCTTGATGAAAATGACTTGGCA...,GAGTTTTTCATCAGTATGTCTGAAACCATTAAATATAATGACGATG...,GTAACTATAACTTTGGATATTAAATCTTGATGAAAATGACTTGGCA...,GTCTGGCGTGCTCCTGATCAGAGTTACTCTGATCTGAAGATTTTGG...
3320,C20orf197,chr20,+,58630979,58648008,58645029,58645963,4,"58630979,58633961,58644887,58645651,","58631063,58634041,58645098,58648008,",...,2357,553,211,13908,ATATCCTGCCTGAGCCTGGGCCGGGTGGAGGTGTCCTGCATGGAGT...,TGTTGGTTAAGGAATCCAGATGTTACTGCAATAACCACTCCATAAA...,GTGAGTCCAGCACACAACTTTCTGACAGTGAACAGGAGTAAACATG...,TGTCAACACTTCAAACAACACAGACAGTACAATGACAAATTGGAGA...,GTGAGTCCAGCACACAACTTTCTGA

### Select positive strand

In [13]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]

In [14]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [15]:
Fasta_Pos

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,Group,Sequence,GeneLength,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,LastIntronSequence,LastExonSequence
0,Gpr39,chr1,+,125676994,125873862,125677336,125872884,2,"125676994,125872369,","125678192,125873862,",...,2,GGGACCTGGAAGGGAGAGGAGGATCCGCCGCCAGACAGAGTTCTGC...,196868,1493,194177,1198,1198,GGGACCTGGAAGGGAGAGGAGGATCCGCCGCCAGACAGAGTTCTGC...,GTGAGTCTAGGTCGCGGTGGGGAGAGGGGGCTGGGGGGAGGAGGGG...,GACTGATTGTGGTGACGTTGGCCGTGTGTTGGATGCCCAATCAGAT...
1,Lmx1a,chr1,+,167689556,167848741,167689774,167846761,8,"167689556,167691980,167791336,167830545,167832...","167689850,167692167,167791569,167830718,167832...",...,2,CAGAGCCGTCCGAGCAGTTCACGGTGGCTCTTCTCTCTCCTCCCAC...,159185,2141,1798,171,155246,CAGAGCCGTCCGAGCAGTTCACGGTGGCTCTTCTCTCTCCTCCCAC...,GTAAGAGGGATTCAGTCCCCAGAGGTCCCTCTCAGTTCACGCTTGT...,GTGCTGAACCTCTTTTCCATGACTTGGATAGTGATGACACATCTCT...
2,A830018L16Rik,chr1,+,11414104,11975901,11414568,11974966,14,"11414104,11511917,11518577,11545153,11562962,1...","11414637,11512065,11518709,11545282,11562991,1...",...,2,GGGTGCTGGTTCCTGGCAGCCAGGAGCTGGGCGCACACCGTGGCGC...,561797,979,2820,57,557998,GGGTGCTGGTTCCTGGCAGCCAGGAGCTGGGCGCACACCGTGGCGC...,GTATGtttgctcagggaatttgtaggtgaattttcccctaatttac...,GTCTGTCTGTGAAACCGGCAAAGGAAGTTGCAAGTGGTCCTTAAGA...
3,Efhc1,chr1,+,20951687,20990841,20951733,20990688,11,"20951687,20955334,20960129,20961152,20967323,2...","20951796,20955556,20960417,20961302,20967516,2...",...,2,AACCCGAGCGGCAAAAGGCAACCGAGGTGGCAGCTGCAGGGCAGCG...,39154,225,1014,235,37915,AACCCGAGCGGCAAAAGGCAACCGAGGTGGCAGCTGCAGGGCAGCG...,GTCAGTGTGGATTGTGGTCAGTGTGGATTGTGGTTTGGCCTGATTG...,TTGATCAGGCTGTGCACCCACGGAGAAGGCAGGATTAACTACTATA...
4,Khdrbs2,chr1,+,32172713,32658568,32172917,32657541,9,"32172713,32243899,32333465,32414870,32467759,3...","32173008,32244027,32333582,32415017,32467887,3...",...,2,GGCCGCACGGGCATCCTCTCCAGCTGAGGCCACGGCCGGAGCCTGG...,485855,1125,13292,59,471438,GGCCGCACGGGCATCCTCTCCAGCTGAGGCCACGGCCGGAGCCTGG...,GTAAGACCGCTTCCTTTCTATGGTCAAGAGATCAGAGCACCCATCT...,CACCAGAAGAATGGGCCACAACTCGCTCCAGCCTGAAGGCACCACC...
5,Gpr45,chr1,+,42952962,43035456,43032198,43033320,2,"42952962,43031951,","42953233,43035456,",...,2,GGTACCCTGGTCCCTAGGGGAGGTCTGCACCCCCGGAGGGTGGCGG...,82494,3505,78718,271,271,GGTACCCTGGTCCCTAGGGGAGGTCTGCACCCCCGGAGGGTGGCGG...,GTAAGTGAccggggccagggccgggccgcggggggcTTGGTGGGGC...,GCTGTAGCATCCTAGGGAGCTGTGCAAGCCCATGGCTACCACAGAA...
6,Dnah7b,chr1,+,46066737,46373546,46066737,46373439,66,"46066737,46066944,46078522,46081434,46083704,4...","46066824,46067004,46078585,46081497,46083813,4...",...,2,ATGCCTCAGAGAGCCACTCTGCTCGACCTCTCAGAAAGTTCTGCTC...,306809,314,10226,104,296269,ATGCCTCAGAGAGCCACTCTGCTCGACCTCTCAGAAAGTTCTGCTC...,GTTAGTATTTACTAAGGGTTACATGCACGTGTGCGCGGATATGGAT...,ATGTGGCTAAAGCCATGCAAAAAGTCAGACATACCCAAACGACCAA...
7,Plcl1,chr1,+,55405920,55754285,55406387,55751463,6,"55405920,55695744,55701929,55713420,55715493,5...","55406630,55698219,55702133,55713496,55715603,5...",...,2,GGAGTGGAGGCCGCCCCGCCGCCGCACTTCCTGGGGCCGCCGTGCT...,348365,3005,35677,110,309683,GGAGTGGAGGCCGCCCCGCCGCCGCACTTCCTGGGGCCGCCGTGCT...,GTACATAAGTTCTCTTGCTGTTTCTTTTGTCATTTCTCTGCTCTTC...,GGACAAGGAGACCTGCTGAAGAATGCCAAGAATGAAGCCGTGGAGA...
8,Bmpr2,chr1,+,59763399,59879014,59764652,59870485,13,"59763399,59813587,59815240,59841430,59842742,5...","59764728,59813758,59815411,59841541,59842834,5...",...,2,AGTGGcgccgccgccgccgccgccgccgccgccCGCGCTGCACCAC...,115615,8780,1619,1280,105216,AGTGGcgccgccgccgccgccgccgccgccgccCGCGCTGCACCAC...,GTAAGTGGAGGGGTTATGCAGTACCCTTTGGGACAGTTAAATAAAA...,TAGGTGAGTCAACACAAGATGGCAAATCAGGATCAGGTGAAAAGAT...
9,Pard3b,chr1,+,61638823,62642284,61639242,62637923,23,"61638823,61767947,62010306,62076384,62144846,6...","61639362,61768049,62010478,62076494,62144935,6...",...,2,GAGGGAGCCCAGAGCGCCGCGCGCGTCGGAGGAGTTGGGAGCCGGC...,1003461,4719,49749,80,948993,GAGGGAGCCCAGAGCGCCGCGCGCGTCGGAGGAGTTGGGAGCCGGC...,GTAGGATATAGAGGGCTCTCCCATCCCCAGATCACCCACAGATGAA...,GGGAGGCTCAGCAGATCCCGTGGACTA

In [16]:
Sequence = Fasta_Pos["Sequence"].tolist()

In [17]:
df = pd.DataFrame(range(-1000,1001),columns=["index"])
for i in range(len(Fasta_Pos)):

    input_sequence = Sequence[i]

    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()

    
    # Define exon location
    Exon = []
    a = Fasta_Pos.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Pos.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)

    # Define probability in cDNA
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)
    
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)        

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative strand

In [18]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]

In [19]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [20]:
Fasta_Neg

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,Group,Sequence,GeneLength,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,LastIntronSequence,LastExonSequence
0,Adora1,chr1,-,134199224,134235431,134202950,134234355,3,"134199224,134234014,134235227,","134203590,134234412,134235431,",...,2,ACTCAGCTCCCGCGCCGGGGGCGACGAAGGCGCCGGCGCCAGAGTG...,36207,4366,30424,398,1417,ACTCAGCTCCCGCGCCGGGGGCGACGAAGGCGCCGGCGCCAGAGTG...,GTGAGTTCATAGCAGCCCAAGGTACTCTGTGCAACCCTGTGTGGGG...,GTACAAGACAGTGGTGACTCAGCGGCGGGCAGCAGTGGCCATAGCC...
1,Sntg1,chr1,-,8361474,9299238,8363474,8803943,21,"8361474,8414202,8445026,8447979,8463393,846543...","8363633,8414313,8445119,8448132,8463465,846555...",...,2,CTTCGCGGCGATACAGCCTCTCGGAAACAAAGGTATCCCTAGCCTT...,937764,2159,50569,111,885036,CTTCGCGGCGATACAGCCTCTCGGAAACAAAGGTATCCCTAGCCTT...,GTAAACACAAGAAACCTCCATATAATACATCTCCAATTATGTCAGT...,GAGTTGGAATTTTCAAATTTATTTGCTGTTCTTCACTGTATTCATT...
2,Adgrb3,chr1,-,25067549,25829707,25068167,25826760,31,"25067549,25074684,25084243,25093764,25098875,2...","25068356,25074789,25084279,25094408,25098915,2...",...,2,GGCTCTGGCCGCTGGATACCACCGCCCCCCCCCCTATTATTTTTTT...,762158,807,6328,105,755023,GGCTCTGGCCGCTGGATACCACCGCCCCCCCCCCTATTATTTTTTT...,GTGAGTATATTCTGCATCCACAATGCCCCAAGCCAAGCTGGGCCAC...,GAAAACCCAGCACCAAACAAGAATCCATGGGACACTTTCAAACCCC...
3,Prim2,chr1,-,33453809,33669795,33454085,33669011,14,"33453809,33464052,33480457,33484681,33512015,3...","33454304,33464121,33480540,33484808,33512201,3...",...,2,CTTCCCGTTCCAAATGAACTCTCCCGCCACTGGGAGAGAGCGGCTG...,215986,495,9748,69,205743,CTTCCCGTTCCAAATGAACTCTCCCGCCACTGGGAGAGAGCGGCTG...,GTAAGTAAATTGTTTAACTTTATAGATTGGTTGTATGTGGCTTGTT...,GTGGACGATTGTGGCTTTTCTTTGAATCATCCAAATCAGTTCTTTT...
4,Pld5,chr1,-,175962305,176275298,175963827,176274874,10,"175962305,175970477,175975458,175984446,175993...","175964084,175970592,175975627,175984583,175994...",...,2,AGTTGGTGCGGAGCAGCCTCGAGCGGAGCTCCCGCTCGGGACCGCA...,312993,1779,6393,115,304821,AGTTGGTGCGGAGCAGCCTCGAGCGGAGCTCCCGCTCGGGACCGCA...,GTAGGTGTGGGGGTTGAGAAGCCCCTCGGGGGTCACAGTAGGGGGA...,GAAATTTTGACTGGGTAGGGAATGACTTTACCCAGAACGCTGGCAC...
5,Syt14,chr1,-,192891232,193035775,192897306,193035635,8,"192891232,192901751,192930477,192933113,192980...","192897620,192901941,192930927,192933385,192980...",...,2,GCCCCGCCCCAGCTCCTGGAGGCCCAGCTGCCTGGCCCCGGGGCGT...,144543,6388,4131,190,134024,GCCCCGCCCCAGCTCCTGGAGGCCCAGCTGCCTGGCCCCGGGGCGT...,GTGAGTGAAAAAGAGTTATTTTTATCTTAATGTTTTCGGCACTTAG...,ATACATATGTTAAGTTAACACTGCTGAATTCCATGGGCCAAGAGAT...
6,Cpa6,chr1,-,10324726,10719945,10325123,10719727,11,"10324726,10329472,10337266,10408268,10409256,1...","10325311,10329557,10337469,10408359,10409367,1...",...,2,GCTTCCAGGCTGGGCTGGGCTGCATTTCCTCACAAGACCCAGTCTT...,395219,585,4161,85,390473,GCTTCCAGGCTGGGCTGGGCTGCATTTCCTCACAAGACCCAGTCTT...,GTAAGTTTCCTGTGCACGTCATATTAGAATTGTGCCATGACTTCCC...,ATGTAAGCTCTGGTAACTCAATGGATTGGGCTTATAAAAATGGAAT...
7,Lonrf2,chr1,-,38794508,38821215,38798028,38821215,11,"38794508,38800650,38804238,38807040,38807835,3...","38798223,38800800,38804401,38807199,38807957,3...",...,2,ATGAAGAACTATGATCAAGCCCTGCAGGATGCTGAAGCAGTTTGTC...,26707,3715,2427,150,20565,ATGAAGAACTATGATCAAGCCCTGCAGGATGCTGAAGCAGTTTGTC...,GTAGGCTTTCCTTCCTTCTTTCCTGGTGCAGCTCCTCCCCTCAGTC...,AGCAATTCTAGTGGCCCTGCCTGGTCCTGGTGGATTCTGGCCGTGT...
8,Dnah7a,chr1,-,53397005,53706761,53397106,53706692,66,"53397005,53405675,53411505,53413731,53419544,5...","53397313,53405779,53411683,53413884,53419775,5...",...,2,CTCCTCTCCAGGTCCTGAGTTCCGCAGCAGCGATCCCGGCCGGCGT...,309756,308,8362,104,301086,CTCCTCTCCAGGTCCTGAGTTCCGCAGCAGCGATCCCGGCCGGCGT...,GTTAGTATTTACTAAGGGTTACATGCACGTGTGTGCGGATATGGAT...,ATGTGGCTAAAGCCATGCAAAAAGTCAGACATACCCAGACGACCAA...
9,Pgap1,chr1,-,54472993,54557682,54480705,54557580,27,"54472993,54481851,54486469,54490220,54492028,5...","54480844,54481956,54486657,54490271,54492164,5...",...,2,AGGCGCGCGCTGCTCAGCTGGGCACGGCTTGGCACCGGAGTCTTTC...,84689,7851,1007,105,75831,AGGCGCGCGCTGCTCAGCTGGGCACGGCTTGGCACCGGAGTCTTTC...,GTACGACAG

In [21]:
Sequence = Fasta_Neg["Sequence"].tolist()

In [22]:
for i in range(len(Fasta_Neg)): 
    
    input_sequence = Sequence[i]
    
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()

    
    # Define exon location
    Exon = []
    a = Fasta_Neg.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Neg.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    Length = Length[::-1]
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)

    # Define probability in cDNA
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)
    
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)        

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [23]:
df

,index,Gpr39,Lmx1a,A830018L16Rik,Efhc1,Khdrbs2,Gpr45,Dnah7b,Plcl1,Bmpr2,...,Calhm1,Calhm3,Obfc1,Xpnpep1,Gpam,Dclre1a,Vax1,Pdzd8,Rab11fip2,Sfxn4
0,-1000,3.410661e-07,9.960404e-10,4.522095e-05,9.874284e-09,1.628747e-08,0.000000e+00,1.243748e-11,1.650154e-10,1.980493e-08,...,0.0,0.0,3.586399e-10,8.881744e-11,1.051486e-09,2.838641e-07,0.0,6.651790e-09,3.535140e-11,2.615876e-12
1,-999,1.553965e-07,3.223006e-09,1.120975e-10,2.418615e-04,5.115084e-09,0.000000e+00,2.661461e-11,1.055594e-08,2.017203e-08,...,0.0,0.0,2.415429e-11,2.263511e-11,3.545020e-05,2.495632e-07,0.0,9.390646e-10,3.356898e-11,2.372894e-12
2,-998,1.530121e-06,1.463316e-08,8.310132e-11,2.065359e-08,6.443544e-11,0.000000e+00,2.903487e-11,1.031167e-08,1.768122e-03,...,0.0,0.0,8.063296e-11,4.901663e-10,9.865229e-10,5.832165e-09,0.0,3.882183e-09,3.725036e-11,5.243514e-12
3,-997,4.031021e-02,1.884429e-08,2.538651e-11,3.167930e-09,6.628977e-11,0.000000e+00,1.204637e-11,2.273915e-09,7.539335e-07,...,0.0,0.0,4.888764e-11,1.206728e-11,2.855645e-11,2.476456e-08,0.0,9.266238e-10,2.242917e-11,1.832595e-10
4,-996,1.787832e-07,1.437113e-09,3.093705e-12,3.532801e-09,6.096118e-10,0.000000e+00,2.609395e-11,1.022763e-09,1.024630e-07,...,0.0,0.0,1.728234e-11,6.390974e-10,1.668099e-09,2.115764e-08,0.0,2.090651e-09,8.354505e-11,1.386252e-11
5,-995,9.481470e-09,1.655192e-08,6.037919e-12,9.071729e-05,1.326088e-09,0.000000e+00,5.621475e-12,2.080592e-09,3.160027e-08,...,0.0,0.0,1.074417e-09,4.098923e-11,2.406074e-10,2.221961e-07,0.0,9.432033e-08,3.808462e-10,9.303265e-12
6,-994,1.171074e-07,2.912713e-09,3.691158e-11,7.446640e-10,1.142202e-10,0.000000e+00,5.843972e-12,3.051843e-08,1.142008e-06,...,0.0,0.0,2.066749e-10,9.513816e-12,2.076795e-10,1.551780e-07,0.0,4.199840e-09,1.408965e-10,1.351011e-10
7,-993,5.480217e-07,1.172541e-09,3.363361e-11,3.861669e-10,3.404300e-11,0.000000e+00,2.968146e-11,2.367728e-09,2.889208e-07,...,0.0,0.0,2.915557e-10,3.583345e-11,2.084177e-10,6.301884e-08,0.0,5.924103e-09,8.081099e-10,2.065893e-10
8,-992,1.218395e-08,4.891917e-08,3.965262e-11,2.268177e-09,6.836415e-11,0.000000e+00,9.018527e-13,1.058769e-09,6.526879e-08,...,0.0,0.0,1.520923e-10,7.838705e-11,7.793340e-11,3.670696e-06,0.0,1.782334e-03,1.824507e-09,1.158569e-05
9,-991,4.279207e-08,2.968143e-02,6.295857e-13,1.774522e-10,5.798946e-10,0.000000e+00,3.935670e-12,1.385849e-08,1.822071e-07,...,0.0,0.0,9.794786e-10,2.475490e-11,9.185561e-11,1.372521e-01,0.0,9.884745e-08,1.174014e-10,3.979430e-10


In [ ]:
df.to_csv("iM6A_mouse_WT_2genes.csv", index=0)

In [24]:
df.drop(["index"], axis=1, inplace=True)
df = df.T

In [25]:
df.loc["Sum"] = df.sum()
df = df.T

In [26]:
Value = df[["Sum"]]

In [27]:
Value.head(5)

,Sum
0,4.283450
1,4.960641
2,6.900568
3,3.857484
4,4.091454


In [28]:
df.drop(["Sum"], axis=1, inplace=True)
df = df.T

In [29]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [30]:
Number = df[["Number"]]

In [31]:
Number.head(5)

,Number
0,6017.0
1,6026.0
2,6031.0
3,6036.0
4,6039.0


In [32]:
Data = pd.concat([Value, Number], axis=1)

In [33]:
Data["Mean"] = Data["Sum"]/Data["Number"]

In [34]:
Data

,Sum,Number,Mean
0,4.283450,6017.0,0.000712
1,4.960641,6026.0,0.000823
2,6.900568,6031.0,0.001144
3,3.857484,6036.0,0.000639
4,4.091454,6039.0,0.000678
5,5.058399,6043.0,0.000837
6,5.440455,6046.0,0.000900
7,5.137514,6049.0,0.000849
8,4.723224,6055.0,0.000780
9,7.538787,6059.0,0.001244


In [36]:
Data = Data.head(2000)
Sum = []
Number = []

for j in range(0,len(Data),10):

    a = Data.loc[j:(j+9),"Sum"]
    b = np.mean(a, axis=0)
    Sum.append(b)
    
    a = Data.loc[j:(j+9),"Number"]
    b = np.mean(a, axis=0)
    Number.append(b)

Result = pd.DataFrame({"Index":range(-1000,1000,10), "Sum":Sum, "Number":Number})
Result["Mean"] = Result["Sum"]/Result["Number"] 

In [37]:
Result

,Index,Number,Sum,Mean
0,-1000,6040.1,5.199198,0.000861
1,-990,6077.7,4.485995,0.000738
2,-980,6124.5,4.608537,0.000752
3,-970,6180.4,4.534830,0.000734
4,-960,6230.2,4.481050,0.000719
5,-950,6276.9,4.616581,0.000735
6,-940,6313.0,4.527727,0.000717
7,-930,6362.3,4.441368,0.000698
8,-920,6416.3,4.593157,0.000716
9,-910,6475.7,4.608915,0.000712


In [38]:
Result.to_csv("iM6A_mouse_WT.csv", index=0)